## Задание 1. Загрузка данных. 
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv ('horse_data.csv', na_values = ['?'], header = None)
df.columns = ['surgery?', 'Age', 'Hospital Number', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'peripheral pulse', 'mucous membranes',
             'capillary refill time', 'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux', 'nasogastric reflux PH', 'rectal examination', 
             'abdomen', 'packed cell volume', 'total protein', 'abdominocentesis appearance', 'abdomcentesis total protein', 'outcome', 'surgical lesion?', 'site of lesion', 
              'type of lesion', 'specific code', 'cp_data']
df_analysis = df [['surgery?', 'Age','rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']]
df_analysis.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


## Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [3]:
#посмотрим часть базовых статистик для выбранных столбцов
df_analysis.describe()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


В столбце "surgery?" нет выбросов, так как может быть только 2 значения и max = 2.0.\
В столбце "Age" предполагается явный выброс, так как максимально должно быть 2.0 по описанию, а здесь max = 9.0\
В столбце "rectal temperature" не видно явных выбросов судя по максимальным и минимальным значениям\
В столбце "pulse" предполагается выброс, так как очень высокое максимальное значение значительно превосходящее 75 процентиль\
Также в столбце "respiratory rate" предполагается выброс, так как очень высокое максимальное значение значительно превосходящее 75 процентиль. Возможно это одна запись вместе с выбросом в столбце "Pulse"\
В столбце "temperature of extremities" не видно явных выбросов судя по максимальным и минимальным значениям
В столбце "pain" не видно явных выбросов судя по максимальным и минимальным значениям\
В столбце "outcome" не видно явных выбросов судя по максимальным и минимальным значениям

In [4]:
# посчитаем еще моду для категориальных значений, это пригодится при работе с пропущенными значениями
print('Значение моды столбца "surgery?": ', df_analysis['surgery?'].round().mode()[0])
print('Значение моды столбца "Age": ', df_analysis['Age'].round().mode()[0])
print('Значение моды столбца "temperature of extremities": ', df_analysis['temperature of extremities'].round().mode()[0])
print('Значение моды столбца "pain": ', df_analysis['pain'].round().mode()[0])
print('Значение моды столбца "outcome": ', df_analysis['pain'].round().mode()[0])

Значение моды столбца "surgery?":  1.0
Значение моды столбца "Age":  1
Значение моды столбца "temperature of extremities":  3.0
Значение моды столбца "pain":  3.0
Значение моды столбца "outcome":  3.0


А теперь найдем выбросы в столбцах 'Age', 'Pulse', 'respiratore rate' через межквартльный размах 

In [5]:
q1 = df_analysis['Age'].quantile(0.25)
q3 = df_analysis['Age'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df_analysis[df_analysis['Age'].between(lower_bound, upper_bound, inclusive=True)].sort_values('Age')
pd.concat([df_analysis, remove_outliers]).drop_duplicates(keep=False)

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
9,2.0,9,38.3,90.0,NaN,1.0,5.0,1.0
13,2.0,9,38.0,92.0,28.0,1.0,1.0,2.0
16,1.0,9,NaN,128.0,36.0,3.0,4.0,2.0
23,1.0,9,38.3,130.0,60.0,NaN,2.0,1.0
39,1.0,9,39.2,146.0,96.0,NaN,NaN,2.0
41,2.0,9,39.0,150.0,72.0,NaN,NaN,1.0
55,1.0,9,38.6,160.0,20.0,3.0,3.0,2.0
74,1.0,9,NaN,NaN,NaN,NaN,NaN,2.0
75,1.0,9,39.7,100.0,NaN,3.0,2.0,3.0


In [6]:
len(pd.concat([df_analysis, remove_outliers]).drop_duplicates(keep=False))

24

Порядка 24 записей в столбце "Age" неверные. При этом визуально не попадался нигде показатель 2в данном столбце, который указан в описании

In [37]:
# проверим наличие показателя 2 в столбце "Age" в датафрейме
df_analysis['Age'].value_counts()

1    276
9     24
Name: Age, dtype: int64

Таким образом, можно предполагать, что скорее всего ошибка внесения данных и вместо 2 было указно значение 9

In [8]:
# посмотрим выбросы столбца pulse
q1 = df_analysis['pulse'].quantile(0.25)
q3 = df_analysis['pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df_analysis[df_analysis['pulse'].between(lower_bound, upper_bound, inclusive=True)].sort_values('pulse')
df_pulse = pd.concat([df_analysis, remove_outliers]).drop_duplicates(keep=False)
df_pulse['pulse'].dropna()

3      164.0
41     150.0
55     160.0
255    184.0
275    150.0
Name: pulse, dtype: float64

выбросами столбца "pulse" являются показатели более 150. Хотя на основании открытых данных можно понять, что это скорее всего реальные данные.

In [9]:
# посмотрим выбросы столбца respiratory rate
q1 = df_analysis['respiratory rate'].quantile(0.25)
q3 = df_analysis['respiratory rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df_analysis[df_analysis['respiratory rate'].between(lower_bound, upper_bound, inclusive=True)].sort_values('respiratory rate')
df_rate = pd.concat([df_analysis, remove_outliers]).drop_duplicates(keep=False)
df_rate['respiratory rate'].dropna()

3      84.0
39     96.0
41     72.0
82     80.0
84     80.0
103    68.0
106    96.0
120    66.0
125    68.0
208    80.0
229    70.0
244    88.0
255    84.0
265    68.0
295    70.0
Name: respiratory rate, dtype: float64

выбросами столбца "respiratory rate" являются показатели более 66. На основании открытых данных можно понять, что это скорее всего реальные значения.

## Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [10]:
# смотрм информацию по столбцам
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


Видно,что в столбце "Age" нет пропусков, а в столбцах "surgery?" и "outcome" по одному отсутствующему значению и много пропусков остальных столбцах

In [11]:
#посмотрим в % сколько пропусков в каждом столбце
(df_analysis.isna().mean() * 100).round(2)

surgery?                       0.33
Age                            0.00
rectal temperature            20.00
pulse                          8.00
respiratory rate              19.33
temperature of extremities    18.67
pain                          18.33
outcome                        0.33
dtype: float64

In [12]:
#посмотрим, что это за строка с пропускоом в стоблце "surgery?"
print(df_analysis[df_analysis['surgery?'].isnull()])

     surgery?  Age  rectal temperature  pulse  respiratory rate  \
132       NaN    1                38.0   48.0              20.0   

     temperature of extremities  pain  outcome  
132                         3.0   4.0      NaN  


Мы сразу нашли и пропущенное значение в столбце "outcome". 
Косвенно судя по контексту и небольшие знания в медицине и хирургии, могу предположить, что показатель боли 4.0 характеризует о серьезной патологии и скорее всего потребовалась операция, поэтому в столбце "surgery?" логично поставить 1.0, а также учитывая достаточно высокие показатели температуры, пульса и дыхания, в столбце исходов "outcome" логично поставить 1.0, с прицелом на благополучный исход. В целом мы фактически заполняем пропуски пользуясь своим знанием предметной области, но как вариант и заполнить модальными значениями. Учитывая, небольшой % пропусков, глобально на анализе это не должно сказаться.

In [13]:
df_analysis['surgery?'].fillna(1.0, inplace = True)

C:\Users\BEU_RU1\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [14]:
df_analysis['outcome'].fillna(1.0, inplace = True)

In [15]:
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    300 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


Руководствуясь своими косвенными знаниями предметной области, я делаю предположение, что ректальная температура, пульс и частота дыхания зависят от возраста и служат прогностическими критериями для исходов (outcome). Считаю логичным посчитать медианные значения для показателей "ректальная температура, пульс и частота дыхания" в группировав по возрасту и исходам, чтобы потом заполнить данными значениями пропуски.

In [16]:
print(df_analysis.groupby(['Age', 'outcome'])['rectal temperature'].median())

Age  outcome
1    1.0        38.2
     2.0        38.0
     3.0        38.0
9    1.0        38.4
     2.0        38.8
     3.0        39.7
Name: rectal temperature, dtype: float64


In [17]:
print(df_analysis.groupby(['Age', 'outcome'])['pulse'].median())

Age  outcome
1    1.0         52.0
     2.0         82.0
     3.0         81.0
9    1.0        120.0
     2.0        141.0
     3.0        100.0
Name: pulse, dtype: float64


In [18]:
print(df_analysis.groupby(['Age','outcome'])['respiratory rate'].median())

Age  outcome
1    1.0        24.0
     2.0        28.0
     3.0        24.0
9    1.0        64.0
     2.0        44.0
     3.0         NaN
Name: respiratory rate, dtype: float64


Таким образом, моя догадка косвенно подтвердилась и можно такими значениями заполнить пропуски в столбцах датафрейма. Учитывая, что столбец 'respiratory rate' показывает большую зависимость от возраста и при учете значения столбца outcome пропуск также останется, логично произвести заполнение медианным значением только с учетом возраста лошади.

In [19]:
# заполняем пропуски в столбце rectal temperature
df_analysis['rectal temperature'].fillna(df_analysis.groupby(['Age', 'outcome'])['rectal temperature'].transform('median'), inplace = True)

In [20]:
# заполняем пропуски в столбце pulse
df_analysis['pulse'].fillna(df_analysis.groupby(['Age', 'outcome'])['pulse'].transform('median'), inplace = True)

In [21]:
# заполняем пропуски в столбце 'respiratory rate'
df_analysis['respiratory rate'].fillna(df_analysis.groupby(['Age'])['respiratory rate'].transform('median'), inplace = True)

Осталось заполнить пропуски в категориальных столбцах temperature of extremities и pain. На мой взгляд они могут быть напрямую связаны со значениями в столбце outcome. Поэтому логично посмотреть значения данных столбцов в зависимости от исходов.

In [31]:
print(df_analysis.groupby('outcome')['temperature of extremities'].median())

outcome
1.0    2.0
2.0    3.0
3.0    3.0
Name: temperature of extremities, dtype: float64


In [32]:
print(df_analysis.groupby('outcome')['pain'].median())

outcome
1.0    3.0
2.0    4.0
3.0    3.0
Name: pain, dtype: float64


Данные показатели незначительно отклоняются от модальных и есть определенная корреляция. На соновании этих данныхя и планирую заполнить проупски в столбцах

In [33]:
df_analysis['temperature of extremities'].fillna(df_analysis.groupby('outcome')['temperature of extremities'].transform('median'), inplace = True)

C:\Users\BEU_RU1\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [34]:
df_analysis['pain'].fillna(df_analysis.groupby('outcome')['pain'].transform('median'), inplace = True)

Таким образом, все пропуски заполнены и в качестве финального штриха, я хочу заменить невверные данные о возрасте лошадей в столбце "Age" - поменять значения 9 на 2

In [35]:
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    300 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal temperature          300 non-null    float64
 3   pulse                       300 non-null    float64
 4   respiratory rate            300 non-null    float64
 5   temperature of extremities  300 non-null    float64
 6   pain                        300 non-null    float64
 7   outcome                     300 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [39]:
df_analysis['Age'].replace(9,2, inplace = True)

C:\Users\BEU_RU1\Anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [40]:
df_analysis['Age'].value_counts()

1    276
2     24
Name: Age, dtype: int64

In [41]:
df_analysis.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,3.0,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,3.0,4.0,2.0


In [42]:
#посмотрим как изменились основные показатели в датафрейме после заполнения пропусков
df_analysis.describe()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,1.396667,1.080000,38.158333,71.940000,29.510000,2.363333,3.000000,1.550000
std,0.490023,0.271746,0.659302,28.211274,16.189613,0.966663,1.199219,0.736642
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.900000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.150000,64.000000,24.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,2.000000,40.800000,184.000000,96.000000,4.000000,5.000000,3.000000


Несильно поменялись основные статистические показатели в датафрейме после заполнения пропусков, за исключением столбца "Age". в котором были сильные выбросы, в итоге исправленные для удобства анализа